In [2]:
# default_exp cloudsearch

# cloudsearch

> a library to trigger aws cloudsearch endpoint

In [2]:
#hide
from nbdev.showdoc import *

In [1]:
#export
import pandas as pd
from pprint import pprint
import boto3

In [4]:
#hide
import pickle, os
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-search-2'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
USER = creden['KEY']
PW = creden['PW']
ACCESS_KEY_ID = USER
SECRET_ACCESS_KEY = PW
os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
# os.environ['DAX_ENDPOINT'] = None
REGION = 'ap-southeast-1'

In [20]:
#export
class Search:
  ''' a search class to return search result'''
  def __init__(self, searchTerm:str, key=None, pw=None, region = 'ap-southeast-1',
               endpoint = '',
               requiredFields = [
                 'villa_category_l1_en',
                 'villa_category_l2_en',
                 'villa_category_l3_en',
                 'villa_category_l4_en'
               ] ):
    self.searchTerm = searchTerm
    self.cloudSearch = boto3.client('cloudsearchdomain' ,
                                    aws_access_key_id=key,
                                    aws_secret_access_key=pw,
                                    region_name=region,
                                    endpoint_url= endpoint)
    self.requiredFields = requiredFields

  def createCriticalColumns(self, df):
    ''' fill in required fields if not exist'''
    for col in self.requiredFields:
      if col not in df:
        df[col] = 'noData'
    return df.fillna('noData')

  def search(self,size = 50):
    return self.returnFullSearch(size=size)
  
  def returnFullSearch(self, queryOptions = '{}', filterQuery = '', size = 50):
    query = self.searchTerm
    if filterQuery: 
      searchResults = self.cloudSearch.search(query = query,
                                            filterQuery = filterQuery,
                                            queryOptions = queryOptions,
                                            size=size)['hits']
    else:
      searchResults = self.cloudSearch.search(query = query,
                                            queryOptions = queryOptions,
                                            size=size)['hits']
    results = []
    items = map(lambda x: x.get('fields'),searchResults.get('hit'))
    items =  map(lambda x: dict(zip(x.keys(),map(lambda y: y[0],x.values()))),items)
    return list(items)

  def sortedSearch(self, queryOptions = '{}', filterQuery = '', size = 1000):
    items = self.returnFullSearch(queryOptions = queryOptions, filterQuery = filterQuery, size = size)
    print(f'raw search result is {pd.DataFrame(items, columns= self.requiredFields)}')
    if not items: return []
    df =  self.sortResultsV2(items)
    output_dict = list(
        df.drop(
            ['isNotFresh', 'cat2Count', 'finalCat'], axis = 1
            ).T.to_dict().values()
        )
    return output_dict

  def sortResultsV2(self, items):
    df = pd.DataFrame(items)
    df = self.createCriticalColumns(df)
    df['isNotFresh'] = df['villa_category_l1_en'] != 'Fresh'
    cat2Count = df.groupby('villa_category_l2_en').count()['pr_code']
    df['cat2Count'] = df['villa_category_l2_en'].apply(lambda x: -cat2Count[x])
    df['finalCat'] = df['villa_category_l4_en'].fillna(df['villa_category_l3_en'])
    return df.sort_values(by=['isNotFresh', 'cat2Count', 'finalCat', 'pr_engname'], ascending= True)

In [30]:
searchEndpoint = 'https://search-villa-cloudsearch-2-4izacsoytzqf6kztcyjhssy2ti.ap-southeast-1.cloudsearch.amazonaws.com'
searcher = Search(searchTerm = 'banana', key=USER, pw= PW , endpoint=searchEndpoint)
result = searcher.sortedSearch(size=1000)
len(list(result))

raw search result is     villa_category_l1_en villa_category_l2_en      villa_category_l3_en  \
0                  Fresh        Fresh Produce  Fruits & Vegetable Local   
1                    NaN                  NaN                       NaN   
2                  Fresh        Fresh Produce                    Bakery   
3                  Fresh        Fresh Produce                    Bakery   
4                  Fresh        Fresh Produce                    Bakery   
..                   ...                  ...                       ...   
450                Fresh        Seafood Fresh                Crustacean   
451                Fresh        Seafood Fresh                Crustacean   
452                Fresh        Seafood Fresh                Crustacean   
453                Fresh               Frozen                   Seafood   
454                Fresh        Seafood Fresh             Local Seafood   

    villa_category_l4_en  
0            Local Fruit  
1                    NaN

455

In [31]:
import sys, json
sys.getsizeof(json.dumps(result))

549306

In [21]:
len(searcher.cloudSearch.search(query = 'pork', size = 1000)['hits']['hit'])

726

In [22]:
import json
boostDict = {
    "fields":["online_category_l1_en"]
}
queryOptions = json.dumps(boostDict)
len(searcher.cloudSearch.search(query = 'dairy', queryOptions = queryOptions, size = 10)['hits']['hit'])

10

In [29]:
searcher2 = Search(searchTerm = 'dairy', key=USER, pw= PW , endpoint=searchEndpoint)
result = searcher2.sortedSearch(queryOptions = queryOptions, size=10)

raw search result is   villa_category_l1_en villa_category_l2_en villa_category_l3_en  \
0                Fresh                Dairy               Cheese   
1                Fresh                Dairy     Butter/Margarine   
2                Fresh                Dairy                 Milk   
3                Fresh                Dairy                 Milk   
4                Fresh                Dairy                 Milk   
5                Fresh                Dairy                 Milk   
6                Fresh                Dairy               Cheese   
7                Fresh                Dairy                 Milk   
8                Fresh                Dairy                 Milk   
9                Fresh                Dairy                 Milk   

   villa_category_l4_en  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN  
5                   NaN  
6                   NaN  
7                   Na